<a href="https://colab.research.google.com/github/twister0316/Public/blob/main/excel_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import os

# ---------------------------------------------------------
# [진단] 현재 코랩에 업로드된 파일 목록을 출력합니다.
# ---------------------------------------------------------
print("📂 현재 업로드된 파일 목록:")
files = os.listdir()
print(files)
print("-" * 30)

# ---------------------------------------------------------
# [설정] 업로드한 파일 이름이 정확해야 합니다.
# ---------------------------------------------------------
file_krx = "krx.xlsx - Sheet1.csv"
file_info = "info.xlsx - Sheet1.csv"

# 파일이 제대로 업로드되었는지 확인
#if not os.path.exists(file_krx) or not os.path.exists(file_info):
#    print("❌ [오류] 파일을 찾을 수 없습니다!")
#    print("👉 왼쪽 폴더 아이콘(📂)을 누르고 파일을 드래그해서 업로드해주세요.")
#    print(f"   (필요한 파일명: {file_krx}, {file_info})")

# 파일 존재 여부 체크
if not os.path.exists(file_krx):
    print(f"❌ 오류: '{file_krx}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")
elif not os.path.exists(file_info):
    print(f"❌ 오류: '{file_info}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")

else:
    print("✅ 파일 확인 완료! 통합 작업을 시작합니다...")

    # 1. 파일 읽기 (인코딩 에러 방지)
    # 한글 엑셀 파일은 utf-8이 아니라 cp949로 저장된 경우가 많습니다.
    def read_csv_safe(path):
        try:
            return pd.read_csv(path, encoding='utf-8')
        except UnicodeDecodeError:
            return pd.read_csv(path, encoding='cp949')

    df_krx = read_csv_safe(file_krx)
    df_info = read_csv_safe(file_info)

    # 2. 종목코드 6자리로 통일 (가장 중요한 키 값)
    # 005930이 5930.0 처럼 숫자로 깨져있을 경우를 대비해 0을 채웁니다.
    def clean_code(x):
        try:
            # 소수점 제거 후 문자열 변환 및 0 채우기
            return str(int(float(x))).zfill(6)
        except:
            return str(x).zfill(6)

    # 컬럼명 공백 제거 (에러 방지)
    df_krx.columns = [c.strip() for c in df_krx.columns]
    df_info.columns = [c.strip() for c in df_info.columns]

    # 키 컬럼 생성
    col_krx = '단축코드' if '단축코드' in df_krx.columns else '종목코드'
    col_info = '종목코드'

    df_krx['키_종목코드'] = df_krx[col_krx].apply(clean_code)
    df_info['키_종목코드'] = df_info[col_info].apply(clean_code)

    # 3. 데이터 합치기 (Outer Join: 합집합)
    # 한쪽에만 있는 기업도 모두 포함시킵니다.
    merged_df = pd.merge(df_krx, df_info, on='키_종목코드', how='outer')

    # 4. 데이터 정리 (비어있는 정보 채우기)
    # KRX 파일의 '한글 종목명'이 비어있으면 Info 파일의 '회사명'을 가져옵니다.
    merged_df['최종_회사명'] = merged_df['한글 종목명'].fillna(merged_df['회사명'])

    # 5. 결과 파일에 남길 컬럼 선택
    final_columns = [
        '키_종목코드',      # 종목코드
        '최종_회사명',      # 회사명
        '시장구분',       # (KRX)
        '소속부',         # (KRX)
        '상장일',         # (KRX)
        '상장주식수',     # (KRX)
        '매출액(원)',     # (Info)
        '대표자',         # (Info)
        '주소',           # (Info)
        '홈페이지',       # (Info)
        '전화번호',       # (Info)
        '설립일'          # (Info)
    ]

    # 존재하는 컬럼만 선택해서 저장 (에러 방지)
    available_cols = [c for c in final_columns if c in merged_df.columns]
    result_df = merged_df[available_cols]

    # 보기 좋게 이름 변경
    result_df.rename(columns={'키_종목코드': '종목코드', '최종_회사명': '회사명'}, inplace=True)

    # 6. 엑셀로 저장
    output_filename = "상장사_통합정보_완료.xlsx"
    result_df.to_excel(output_filename, index=False)

    print("\n" + "="*50)
    print(f"🎉 통합 완료! 총 {len(result_df)}개 기업이 정리되었습니다.")
    print(f"📂 왼쪽 파일 탭에서 '{output_filename}' 파일을 우클릭하여 다운로드하세요.")
    print("="*50)

📂 현재 업로드된 파일 목록:
['.config', '.ipynb_checkpoints', 'krx.xlsx', 'info.xlsx', 'sample_data']
------------------------------
❌ 오류: 'krx.xlsx - Sheet1.csv' 파일을 찾을 수 없습니다.
👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.


In [19]:
import pandas as pd
import os

# ---------------------------------------------------------
# [설정] 파일 이름
# ---------------------------------------------------------
file_krx = "krx.csv"
file_info = "info.csv"

# 파일 존재 확인
if not os.path.exists(file_krx) or not os.path.exists(file_info):
    print("❌ 파일을 찾을 수 없습니다.")
    print(f"📂 현재 파일 목록: {os.listdir()}")
else:
    print("✅ 파일 확인 완료! 통합을 시작합니다...")

    # =========================================================
    # [핵심] 강력한 파일 읽기 함수 (ParserError 해결)
    # =========================================================
    def read_csv_robust(path):
        # 1. 인코딩 리스트 (한국어 파일 호환)
        encodings = ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']

        for enc in encodings:
            try:
                # engine='python': 콤마나 따옴표 문제를 더 잘 해결함
                # on_bad_lines='skip': 형식이 깨진 불량 행은 무시하고 넘어감
                df = pd.read_csv(path, encoding=enc, engine='python', on_bad_lines='skip')
                print(f"  -> 성공! 읽기 완료: {enc} ({path})")
                return df
            except Exception as e:
                # 실패하면 다음 인코딩 시도
                continue

        # 2. CSV로 안 읽히면 엑셀(XLSX)로 읽기 시도 (혹시 확장자만 csv인 엑셀파일일 경우)
        try:
            print(f"  -> CSV 읽기 실패. 엑셀 형식으로 시도합니다... ({path})")
            return pd.read_excel(path)
        except:
            pass

        print(f"❌ 읽기 실패: {path} (파일이 손상되었거나 형식이 맞지 않습니다.)")
        return pd.DataFrame() # 빈 데이터프레임 반환

    # 파일을 읽어옵니다
    print("📂 파일 읽기 시도 중...")
    df_krx = read_csv_robust(file_krx)
    df_info = read_csv_robust(file_info)

    # 데이터가 비어있으면 중단
    if df_krx.empty or df_info.empty:
        print("❌ 데이터를 읽어오지 못했습니다. 파일을 다시 확인해주세요.")
    else:
        # ---------------------------------------------------------
        # 데이터 통합 로직
        # ---------------------------------------------------------

        # 종목코드 6자리 통일
        def clean_code(x):
            try:
                return str(int(float(x))).zfill(6)
            except:
                return str(x).zfill(6)

        # 컬럼 공백 제거
        df_krx.columns = [c.strip() for c in df_krx.columns]
        df_info.columns = [c.strip() for c in df_info.columns]

        # 키 컬럼 생성
        col_krx = '단축코드' if '단축코드' in df_krx.columns else '종목코드'
        col_info = '종목코드'

        # 키 생성
        if col_krx in df_krx.columns:
            df_krx['키_종목코드'] = df_krx[col_krx].apply(clean_code)

        if col_info in df_info.columns:
            df_info['키_종목코드'] = df_info[col_info].apply(clean_code)

        # 병합 (Outer Join)
        if '키_종목코드' in df_krx.columns and '키_종목코드' in df_info.columns:
            merged_df = pd.merge(df_krx, df_info, on='키_종목코드', how='outer')

            # 회사명 합치기
            name_col_krx = '한글 종목명' if '한글 종목명' in merged_df.columns else '회사명_x'
            name_col_info = '회사명' if '회사명' in merged_df.columns else '회사명_y'

            # 회사명 컬럼 찾기 및 병합
            cols = merged_df.columns
            final_name_col = name_col_krx if name_col_krx in cols else (name_col_info if name_col_info in cols else None)

            if name_col_krx in cols and name_col_info in cols:
                merged_df['최종_회사명'] = merged_df[name_col_krx].fillna(merged_df[name_col_info])
            elif final_name_col:
                merged_df['최종_회사명'] = merged_df[final_name_col]
            else:
                merged_df['최종_회사명'] = "확인불가"

            # 저장할 컬럼 정리
            final_columns = [
                '키_종목코드', '최종_회사명', '시장구분', '소속부', '상장일', '상장주식수',
                '매출액(원)', '대표자', '주소', '홈페이지', '전화번호', '설립일'
            ]

            available_cols = [c for c in final_columns if c in merged_df.columns]
            result_df = merged_df[available_cols]
            result_df.rename(columns={'키_종목코드': '종목코드', '최종_회사명': '회사명'}, inplace=True)

            # 엑셀 저장
            output_name = "상장사_통합정보_완료.xlsx"
            result_df.to_excel(output_name, index=False)

            print("\n" + "="*50)
            print(f"🎉 성공! 에러 없이 '{output_name}' 파일이 생성되었습니다.")
            print("="*50)
        else:
            print("❌ '종목코드' 컬럼을 찾을 수 없어 병합하지 못했습니다.")

✅ 파일 확인 완료! 통합을 시작합니다...
📂 파일 읽기 시도 중...
  -> CSV 읽기 실패. 엑셀 형식으로 시도합니다... (krx.csv)
  -> CSV 읽기 실패. 엑셀 형식으로 시도합니다... (info.csv)


/tmp/ipython-input-1086739235.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={'키_종목코드': '종목코드', '최종_회사명': '회사명'}, inplace=True)



🎉 성공! 에러 없이 '상장사_통합정보_완료.xlsx' 파일이 생성되었습니다.
